# Fiddler Performance Metrics Fetcher

This script fetches performance metrics for binary classification models from the Fiddler API.

**Instructions:**
1. Edit the configuration variables below with your API details
2. Run all cells sequentially
3. The script will generate `performance_metrics.csv` with your metrics

**API Endpoint Used:** `/v3/analytics/metrics` with POST request
**Required Parameters:** model_id, env_type, metrics (list of metric objects with 'id' field)

## Configuration
Edit these variables with your Fiddler API details:

In [ ]:
# Configuration - EDIT THESE VALUES
API_URL = ''  # Your Fiddler base URL
API_TOKEN = ''                   # Your API authentication token
MODEL_ID = ''                     # Target model ID
DATA_SOURCE = ''  # Data source: PRODUCTION, BASELINE, or VALIDATION

## Import Dependencies

In [ ]:
import requests
import pandas as pd
import json
from typing import Dict, Any, Optional

## Fetch Performance Metrics

In [ ]:
def fetch_performance_metrics(api_url: str, api_token: str, model_id: str, data_source: str = 'PRODUCTION') -> Optional[Dict[str, float]]:
    """
    Fetch performance metrics from Fiddler API for binary classification model.
    
    Args:
        api_url: Base URL for Fiddler API
        api_token: API authentication token
        model_id: Target model ID
        data_source: Data source (PRODUCTION, BASELINE, or VALIDATION)
        
    Returns:
        Dictionary with performance metrics or None if failed
    """
    
    # Headers for authentication
    headers = {
        'Authorization': f'Bearer {api_token}',
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    
    # API endpoint for analytics metrics
    endpoint = f'{api_url.rstrip("/")}/v3/analytics/metrics'
    
    # Payload for analytics metrics
    payload = {
        'model_id': model_id,
        'env_type': data_source,  # Use env_type instead of data_source
        'metrics': [
            {'id': 'accuracy'},
            {'id': 'precision'}, 
            {'id': 'recall'},
            {'id': 'f1_score'},
            {'id': 'auc'}
        ]
    }
    
    try:
        print(f"Fetching metrics for model: {model_id}")
        print(f"Data source: {data_source}")
        print(f"API endpoint: {endpoint}")
        # Make API request - POST for analytics query
        response = requests.post(endpoint, json=payload, headers=headers, timeout=30)
        
        if response.status_code == 200:
            data = response.json()
            print("✅ Successfully fetched metrics from API")
            return data.get('data', data)  # Try 'data' key first, fallback to full response
        else:
            print(f"❌ API request failed: {response.status_code}")
            print(f"Response: {response.text}")
            return None
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Network error: {str(e)}")
        return None
    except json.JSONDecodeError as e:
        print(f"❌ JSON decode error: {str(e)}")
        return None
    except Exception as e:
        print(f"❌ Unexpected error: {str(e)}")
        return None

In [ ]:
def process_and_save_metrics(metrics_data: Any) -> bool:
    """
    Process metrics data and save to CSV file.
    
    Args:
        metrics_data: Raw metrics data from API (list of values)
        
    Returns:
        True if successful, False otherwise
    """
    
    # The API returns metrics in the order we requested them
    metric_names = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC']
    
    # Handle both list format and dict format
    if isinstance(metrics_data, list):
        if len(metrics_data) != len(metric_names):
            print(f"❌ Expected {len(metric_names)} metrics, got {len(metrics_data)}")
            return False
        required_metrics = dict(zip(metric_names, metrics_data))
    else:
        # Fallback for dict format
        required_metrics = {
            'Accuracy': metrics_data.get('accuracy'),
            'Precision': metrics_data.get('precision'), 
            'Recall': metrics_data.get('recall'),
            'F1 Score': metrics_data.get('f1_score'),
            'AUC': metrics_data.get('auc')
        }
    
    # Check which metrics are available
    available_metrics = {}
    missing_metrics = []
    
    for metric_name, value in required_metrics.items():
        if value is not None:
            available_metrics[metric_name] = value
        else:
            missing_metrics.append(metric_name)
    
    if missing_metrics:
        print(f"⚠️  Warning: Missing metrics: {', '.join(missing_metrics)}")
    
    if not available_metrics:
        print("❌ No performance metrics found in API response")
        return False
    
    # Create DataFrame
    df = pd.DataFrame([
        {'Metric': metric, 'Value': value} 
        for metric, value in available_metrics.items()
    ])
    
    # Save to CSV
    output_file = 'performance_metrics.csv'
    try:
        df.to_csv(output_file, index=False)
        print(f"✅ Performance metrics saved to {output_file}")
        print(f"📊 Saved {len(available_metrics)} metrics:")
        for metric, value in available_metrics.items():
            print(f"   • {metric}: {value:.4f}")
        return True
        
    except Exception as e:
        print(f"❌ Error saving CSV: {str(e)}")
        return False

## Execute Script

In [ ]:
def main():
    """Main execution function"""
    
    print("🚀 Starting Fiddler Performance Metrics Fetcher")
    print("=" * 50)
    
    # Validate configuration
    if (API_URL == 'https://your-fiddler-instance.com' or 
        API_TOKEN == 'your_api_token' or 
        MODEL_ID == 'your_model_id'):
        print("❌ Please edit the configuration variables at the top of this script:")
        print("   • API_URL: Your Fiddler instance URL")
        print("   • API_TOKEN: Your API authentication token") 
        print("   • MODEL_ID: Your target model ID")
        print("   • DATA_SOURCE: Environment type (PRODUCTION, BASELINE, or VALIDATION)")
        return
    
    # Fetch metrics from API
    metrics_data = fetch_performance_metrics(API_URL, API_TOKEN, MODEL_ID, DATA_SOURCE)
    
    if metrics_data is None:
        print("❌ Failed to fetch metrics from API")
        return
    
    # Process and save metrics
    success = process_and_save_metrics(metrics_data)
    
    if success:
        print("\n🎉 Script completed successfully!")
        print("📄 Check performance_metrics.csv for your results")
    else:
        print("\n❌ Script failed to complete")

# Execute main function
if __name__ == "__main__":
    main()